In [13]:
import requests
from bs4 import BeautifulSoup
import os

In [2]:
'''
爬取目標：
json:
[
食譜名稱,
食材 [{"食材":"用量"}]
料理圖片一張,
頁面網址,
菜式分類
]
'''

'\n爬取目標：\njson:\n[\n食譜名稱,\n食材 [{"食材":"用量"}]\n料理圖片一張,\n頁面網址,\n菜式分類\n]\n'

In [3]:
# 取得目標網頁，將 response 的 html 用 bs4 解析轉成 BeautifulSoup 物件 
# 若不是mac，記得要在 Chrome 開發人員工具的 Network 中 Request Headers 找自己的 user-agent
url = "https://icook.tw/categories"
headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
          }
resp = requests.get(url, headers=headers)
html = BeautifulSoup(resp.text)


In [7]:
# 需跳過要vip會員的食譜
base_url = "https://icook.tw"
category = html.find_all("div", class_="inner-block")[2]


#清除不相關的標籤，回傳subcast = [("分類","下一頁的url")]
subcat = []
for sub in category.find_all("a"):
    try:
        pair = (sub["name"], base_url + sub["href"])
        subcat.append(pair)
    except:
        pass  
subcat

[('家常美味', 'https://icook.tw/categories/592'),
 ('烘焙專區', 'https://icook.tw/categories/593'),
 ('烤肉', 'https://icook.tw/categories/104'),
 ('炸物', 'https://icook.tw/categories/491'),
 ('焗烤', 'https://icook.tw/categories/113'),
 ('涼拌', 'https://icook.tw/categories/10'),
 ('沙拉', 'https://icook.tw/categories/7'),
 ('醃漬', 'https://icook.tw/categories/591'),
 ('滷肉', 'https://icook.tw/categories/209'),
 ('麻辣', 'https://icook.tw/categories/217'),
 ('滷味', 'https://icook.tw/categories/207'),
 ('咖哩', 'https://icook.tw/categories/107'),
 ('披薩', 'https://icook.tw/categories/112'),
 ('義大利麵', 'https://icook.tw/categories/111'),
 ('壽司手捲', 'https://icook.tw/categories/108'),
 ('肉丸子', 'https://icook.tw/categories/413'),
 ('熱炒', 'https://icook.tw/categories/208'),
 ('鍋物', 'https://icook.tw/categories/241'),
 ('煎餅', 'https://icook.tw/categories/380'),
 ('常備菜', 'https://icook.tw/categories/596'),
 ('素食', 'https://icook.tw/categories/28'),
 ('全素', 'https://icook.tw/categories/97'),
 ('奶蛋素', 'https://icook.tw/

In [8]:
len(subcat)

191

In [9]:
# 進入每個次分類頁面
recipe_url_list = []
# subcat共191個頁面，光食譜清單就要爬很久，測試時記得把清單縮小 e.g. subcat[:5]
for each_subcat in subcat[:5]:
    bsObj = BeautifulSoup(requests.get(each_subcat[1], headers=headers).text)
    
    # 蒐集所有次分類內的食譜清單
    for each_recipe in bsObj.find_all("a", class_="browse-recipe-touch-link"):
        recipe_url = base_url + each_recipe["href"]
        recipe_url_list.append(recipe_url)
    try:
        if not bsObj.find("a", rel="next")["href"] == None:
            next_pg_html = BeautifulSoup(requests.get(base_url + bsObj.find("a", rel="next")["href"], headers=headers).text)
            getRecipesByPage(next_pg_html)
    except:
        continue

recipe_url_list

['https://icook.tw/recipes/282274',
 'https://icook.tw/recipes/282081',
 'https://icook.tw/recipes/281549',
 'https://icook.tw/recipes/281544',
 'https://icook.tw/recipes/282029',
 'https://icook.tw/recipes/281449',
 'https://icook.tw/recipes/273350',
 'https://icook.tw/recipes/281227',
 'https://icook.tw/recipes/281224',
 'https://icook.tw/recipes/279840',
 'https://icook.tw/recipes/279768',
 'https://icook.tw/recipes/279759',
 'https://icook.tw/recipes/281236',
 'https://icook.tw/recipes/279389',
 'https://icook.tw/recipes/279198',
 'https://icook.tw/recipes/278533',
 'https://icook.tw/recipes/278654',
 'https://icook.tw/recipes/278123',
 'https://icook.tw/recipes/277385',
 'https://icook.tw/recipes/277668',
 'https://icook.tw/recipes/277559',
 'https://icook.tw/recipes/277494',
 'https://icook.tw/recipes/277381',
 'https://icook.tw/recipes/276356',
 'https://icook.tw/recipes/216486',
 'https://icook.tw/recipes/170303',
 'https://icook.tw/recipes/169048',
 'https://icook.tw/recipes/1

In [10]:
len(recipe_url_list)

60

In [28]:
# 檢查是否有重複網址
unique_recipe_url_list = list(set(recipe_url_list))
len(unique_recipe_url_list)

59

In [29]:
# 把做完檢查重複網址的 list 重新參照回去
recipe_url_list = unique_recipe_url_list

In [30]:
# 進入每個食譜
# 測試時記得把清單縮小 e.g. recipe_url_list[:5]
recipe_info = []
for each_recipe_url in recipe_url_list[:10]:
    recipe_html = BeautifulSoup(requests.get(each_recipe_url, headers=headers).text)
    
    # 取得食譜名、食譜圖片網址、材料清單
    # recipe_name : str
    recipe_name = recipe_html.find("h1").text.strip()

    # img_src : str
    # 要存進資料庫時要記得處理IO
    recipe_img_src = recipe_html.find("img", class_="main-pic")["src"]

    # ingredients : dict
    ingredients = recipe_html.find_all("div", itemprop="ingredients")
    ingredient_preparation = []
    for ingre in ingredients:
        ingre_name = ingre.find("div", class_="ingredient-name").text
        ingre_unit = ingre.find("div", class_="ingredient-unit").text
        ingredient_preparation.append({ingre_name:ingre_unit})
    recipe_info.append([recipe_name, recipe_img_src, ingredient_preparation,each_recipe_url])

recipe_info

[['起司豬排',
  'https://tokyo-kitchen.icook.network/uploads/recipe/cover/265378/large_fbc88196a31f6d9b.jpg',
  [{'梅花火鍋豬肉片': '30片'},
   {'切達起司': '5-6片'},
   {'麵粉': '適量'},
   {'蛋': '2顆'},
   {'麵包粉': '適量'},
   {'炸油': '適量'}],
  'https://icook.tw/recipes/265378'],
 ['不用麵粉的披薩',
  'https://tokyo-kitchen.icook.network/uploads/recipe/cover/276730/large_ae517d063617da67.jpg',
  [{'花椰菜': '1顆'},
   {'臘腸': '3片'},
   {'鳳梨': '1片'},
   {'火腿': '3片'},
   {'蘑菇': '適量'},
   {'小番茄': '適量'},
   {'雞蛋': '1顆'},
   {'莫札瑞拉起司絲': '70克'},
   {'義式番茄醬': '1大匙'},
   {'鹽巴': '1/2茶匙'},
   {'黑胡椒': '1/2茶匙'},
   {'起司片': '3片'},
   {'起司粉': '適量'}],
  'https://icook.tw/recipes/276730'],
 ['日式炒烏龍',
  'https://tokyo-kitchen.icook.network/uploads/recipe/cover/279759/large_42ed7d80b942b7b9.jpg',
  [{'洋蔥': '100克'},
   {'紅蘿蔔': '50克'},
   {'高麗菜': '100克'},
   {'日式醬油': '10克'},
   {'米酒': '3克'},
   {'高湯': '200克'},
   {'烏龍麵': '200克'},
   {'烏醋': '2克'},
   {'黑胡椒粒': '2克'}],
  'https://icook.tw/recipes/279759'],
 ['炸魚餅',
  'https://tokyo-kitchen.ico

In [22]:
# 影像儲存測試
img_url = requests.get(recipe_info[0][1])
img_file_name = recipe_info[0][0] + ".jpg"
if not os.path.isdir("img"):
    os.mkdir("img")
with open("img/%s" %img_file_name, "wb") as img:
    img.write(img_url.content)

In [ ]:
# 單頁食譜清單測試
bsObj = BeautifulSoup(requests.get(subcat[0][1], headers=headers).text)


In [ ]:
bsObj.find("a", rel="next")["href"]

In [ ]:
for each_recipe in bsObj.find_all("a", class_="browse-recipe-touch-link"):
    recipe_html = BeautifulSoup(requests.get(base_url + each_recipe["href"], headers=headers).text)
    recipe_html

# bullet = bsObj.find_all("div", class_="categories-browse-recipe")
# name = bsObj.find_all("div", class_="browse-recipe-content")[0]
# recipe_name_list = bsObj.find_all("a", class_="browse-recipe-name")
# recipe_name_list = bsObj.find_all("a", class_="browse-recipe-name")

# recipes = []
# for recipe_name in recipe_name_list:
#     recipes.append(recipe_name["title"])
# recipes



In [ ]:
test_url = base_url + bsObj.find_all("a", class_="browse-recipe-touch-link")[0]["href"]

pg = BeautifulSoup(requests.get(test_url, headers=headers).text)


In [ ]:
# recipe_name : str
recipe_name = pg.find("h1").text.strip()

# img_src : str
img_src = pg.find("img", class_="main-pic")["src"]

# ingredients : dict
ingredients = pg.find_all("div", itemprop="ingredients")
ingredient_preparation = []
for ingre in ingredients:
    ingre_name = ingre.find("div", class_="ingredient-name").text
    ingre_unit = ingre.find("div", class_="ingredient-unit").text
    ingredient_preparation.append({ingre_name:ingre_unit})

In [ ]:
a = "https://icook.tw/recipes/170303"
test = BeautifulSoup(requests.get(a, headers=headers).text)
test

In [ ]:
test.find_all("div", itemprop="ingredients")